In [153]:
import json
import pandas as pd
import numpy as np
import re

In [154]:
#####################################################################################################################

In [155]:
#8.2.1 EXTRACT WIKIPEDIA MOVIE JSON
file_dir = '/Users/josephbassil/Desktop/UC_Berkeley_Aug/Module_8_Movies_ETL/Resources'

In [156]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [157]:
#TO:
#Check first five records in a list (list of dicts):wiki_movies_raw[:5]  
#Check last five records in a list (list of dicts): wiki_movies_raw[-5:]
#Check some records in the middle: wiki_movies_raw[3600:3605]
#NB:To check first/last five in a df: df.head()/df.tail()

In [158]:
#####################################################################################################################

In [159]:
#8.2.2 EXTRACT KAGGLE DATA

In [160]:
kaggle_metadata=pd.read_csv('movies_metadata.csv',low_memory=False)
ratings=pd.read_csv('ratings.csv')

In [161]:
ratings.sample(n=5)
#We are used to using .head() or .tail() to check on our data
#.sample(n=) picks 5 different rows and displays it

,userId,movieId,rating,timestamp
3231966,33608,4025,0.5,1470062039
22524238,233912,2826,3.0,967868172
20149415,209536,3479,4.0,1026917156
3738395,38976,474,4.0,949270700
16359657,170031,1625,4.5,1129711188


In [162]:
#We just completed the Extract part of ETL
#we have:
#wiki_movies_raw (json)
#kaggle_metadata (csv for movies)
#ratings

In [163]:
##############################################################################################################
##############################################################################################################

In [164]:
#8.3.1 Data Cleaning: Inspect, Plan, Execute

In [165]:
#if we inspect wiki_movies_df=pd.DataFrame(wiki_movies_raw), the DataFrame is too Big to inspect 
#and is not fully displated so let's transform it to a list: wiki_movies_df.columns.to_list()
#Now we can inspect our list and check for the unneeded or unnecessary columns
#wiki_movies_df=pd.DataFrame(wiki_movies_raw)
#wiki_movies_df.columns.to_list()
#Now that we have the list, let's clean it and keep what we want

In [166]:
#Now that we have the list, let's clean it and keep what we want
#wiki_movies=[movie for movie in wiki_movies_raw
#            if ('Director' in movie or 'Directed by' in movie
#               and 'imbd_link' in movie
#               and 'No. of episodes' not in movie)]
#wiki_df=pd.DataFrame(wiki_movies)

In [167]:
#That was the first cycle in inspecting your data
#inspecting wiki_df again, we found that it needs another cycle, we need to create another plan of attack

In [168]:
#In general, earlier iterations try to handle big chunks of data at one time, 
#such as removing columns and rows, while later iterations focus on 
#smaller chunks of data, such as parsing values.

In [169]:
#1-LET'S TAKE A CLOSER LOOK AT THE INSPECT STEP
##How many rows and columns, missing values
#check data type: ex: TRUE/FALSE column should have a boolean data type
#Numeric type: central tendency, mean, median, spread, min, max
#We can alos investigate columns with statistical plots, like scatter plots and histograms
#inspect head, tail and sample
#does it have a consistent strcuture (csv) or unstructured (collection of emails)
#how is each data pt identified (explicit, unique ID?)


#2- ONCE WE INVESTIGATED OUR DATA, IT'S TIME TO PLAN AND DECIDE WHAT TO DO
#wrong data type for specific rows or entire column
#do rows have outliers?
#when values are missing, should we remove, replace or interpolate?

#THERE ARE 2 MAIN WAYS: we can Modify values or Structure
    #Modifying values: removing rows, columns, replacing values, generating new columns. remove rows with missing 
    #or corrupted data, columns with one value, columns mostly missing data
    #instead of dropping data, we might replace with zeros or empty strings, we might have a column that contains 
    #nonstandard values, such as percentages that are stored as whole numbers from 0 to 100 and also as fractions 
    #from 0 to 1, and we would replace them with one standard form
    #We can round, bracket, splitting columns
    
    #Modifying data structure: pivoting the values of one column into multiple columns, aggregation rows, merging 
    #multiple datasets
    
#3- EXECUTE
#As you implement your changes, try to take into account any unintended consequences you could introduce.
#we might discover that the problem should be tackled in another way
#Always, come back and and inspect the data in a new iteration
#you might follow a plan, then realize you need to inspect more; executing a plan, then reale a step was missed 
#and you need to quickly rework the plan

In [170]:
#####################################################################################################################

In [171]:
#8.3.3 Investigate the Wikipedia Data
wiki_movies_df=pd.DataFrame(wiki_movies_raw)
wiki_movies_df.columns.to_list() #These are the columns headers
#Let's modify our JSON data by restricting it to only those entries that have a director and an IMDb link.

['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Also known as',
 'Opening theme',
 'No. of episodes',
 'Preceded by',
 'Author',
 'Publisher',
 'Publication date',
 'Media type',
 'Pages',
 'ISBN',
 'OCLC',
 'LC Class',
 'Cover artist',
 'Series',
 'Set in',
 'Adaptation by',
 'Suggested by',
 'Biographical data',
 'Born',
 'Died',
 'Resti

In [172]:
#List Comprehensions to Filter Data
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
len(wiki_movies)
#So now we have cut down the number of movies to 7080

7080

In [173]:
wiki_movies_df=pd.DataFrame(wiki_movies)
wiki_movies_df

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Hepburn,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,https://en.wikipedia.org/wiki/Holmes_%26_Watson,2018,https://www.imdb.com/title/tt1255919/,Holmes & Watson,Etan Cohen,"[Will Ferrell, Adam McKay, Jimmy Miller, Clayt...",Etan Cohen,NaN,"[Sherlock Holmes, and, Dr. Watson, by, Sir Art...","[Will Ferrell, John C. Reilly, Rebecca Hall, R...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7076,https://en.wikipedia.org/wiki/Vice_(2018_film),2018,https://www.imdb.com/title/tt6266538/,Vice,Adam McKay,"[Brad Pitt, Dede Gardner, Jeremy Kleiner, Kevi...",NaN,NaN,NaN,"[Christian Bale, Amy Adams, Steve Carell, Sam ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7077,https://en.wikipedia.org/wiki/On_the_Basis_of_Sex,2018,https://www.imdb.com/title/tt4669788/,On the Basis of Sex,Mimi Leder,Robert W. Cort,NaN,NaN,NaN,"[Felicity Jones, Armie Hammer, Justin Theroux,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7078,https://en.wikipedia.org/wiki/Destroyer_(2018_...,2018,https://www.imdb.com/title/tt7137380/,Destroyer,Karyn Kusama,"[Fred Berger, Phil Hay, Matt Manfredi]",NaN,NaN,NaN,"[Nicole Kidman, Sebastian Stan, Toby Kebbell, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
#it looks like we have some tv shows so let's get rid of those by adding a condition to our wiki_movies
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [175]:
#######################################################################################################################

In [176]:
#8.3.4 FUNCTIONS REVIEW

#REWIND: Functions are code blocks performing specific tasks:
    #4 basic parts: Name, parameters, code block, return
    #scope: Hierarchyof variables: global variable (created outside the function) in our code 
    #or local variables (created inside the function), local variables are only known inside the function
    
    #Keep this trick in mind with the scope of variables: We can create a local variable with the same name as a 
    #global variable. Inside the function, the local variable takes precedence, but the value of the global variable 
    #will be unchanged
    
#    x = 'global value'

#    def foo():
#    x = 'local value'
#    print(x)

#    foo()
#    print(x)

#The output: 
#local value
#global value
    
#When passing mutable objects like a dict or list as parameters to a function, 
#the function can change the values inside the object.
#my_list = [1,2,3]
#def append_four(x):
#    x.append(4)
#append_four(my_list)
#print(my_list)

#the output would be:
#   [1,2,3,4]
#it changed the list even when outside the function, to leave the original object unchanged, we need to make a copy
#to make a copy of the list: new_list=list(old_list)
#                            new_dict=dict(old_dict)


##LAMBDA FUNCTION
#lambda functions have no name
#can be used as once time use function
#ex: square=lambda x: x*x
#    square(5)
#The output will be: 25

In [177]:
#######################################################################################################################

In [178]:
#8.3.5 Create a function to clean data PART 1

In [179]:
def clean_movie(movie):
    movie=dict(movie)
    return movie
#we want to make nondestructive edits to the movies dict, so we need to make a copy of the incoming movie
#to make a copy we'll use dict() constructor: special functions that initialize new objects.
#so when we pass movie as a parameter to dict() constructor, it reserves a new space in memory and copies all
#the info in movie to that new space
#with movie=dict(movie), any changes we make inside the function will now onlt affect the copy, so if we make a mistake
#we still have the original, untouched movie to reference

In [180]:
#let's see which movies are in arabic
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

6838    https://en.wikipedia.org/wiki/The_Insult_(film)
7062     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [181]:
#value_counts() is a quick easy way to see what non-null values there are in a column

In [182]:
sorted(wiki_movies_df.columns)

['Adaptation by',
 'Also known as',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Camera setup',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'No. of episodes',
 'Opening theme',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russ

In [183]:
def clean_movie(movie):
    movie=dict(movie) #create a non-destructive copy
    alt_titles={} #empty dict to hold all the alternative titles
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key]=movie[key]
            movie.pop(key)
        if len(alt_titles)>0:
            movie['alt_titles']=alt_titles
            
        #merge column names
        def change_column_name(old_name,new_name):
            if old_name in movie:
                movie[new_name]=movie.pop(old_name)
            change_column_name('Adaptation by', 'Writer(s)')
            change_column_name('Country of origin', 'Country')
            change_column_name('Directed by', 'Director')
            change_column_name('Distributed by', 'Distributor')
            change_column_name('Edited by', 'Editor(s)')
            change_column_name('Length', 'Running time')
            change_column_name('Original release', 'Release date')
            change_column_name('Music by', 'Composer(s)')
            change_column_name('Produced by', 'Producer(s)')
            change_column_name('Producer', 'Producer(s)')
            change_column_name('Productioncompanies ', 'Production company(s)')
            change_column_name('Productioncompany ', 'Production company(s)')
            change_column_name('Released', 'Release Date')
            change_column_name('Release Date', 'Release date')
            change_column_name('Screen story by', 'Writer(s)')
            change_column_name('Screenplay by', 'Writer(s)')
            change_column_name('Story by', 'Writer(s)')
            change_column_name('Theme music composer', 'Composer(s)')
            change_column_name('Written by', 'Writer(s)')
        return movie

In [184]:
#We can make a list of cleaned movies with a list comprehension
clean_movies=[clean_movie(movie) for movie in wiki_movies]

#Let's set the wiki_movies_df to be the df created from clean_movies and print out a list of columns:
wiki_movies_df=pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Arabic',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cantonese',
 'Chinese',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'French',
 'Genre',
 'Hangul',
 'Hebrew',
 'Hepburn',
 'Japanese',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Mandarin',
 'McCune–Reischauer',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Polish',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Revised Romanization',
 'Romanized',
 'Running time',
 'Russian',
 'Screen story by',
 'Screenplay by',
 'Simplified',
 'Starring',
 

In [185]:
######################################################################################################################

In [186]:
#8.3.7 Remove duplicate rows
#The code to extract the IMDb ID with (r'(tt\d{7})) as a regular expression or regex
#wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
#now we can drop any duplicates od IMDb IDs with drop_duplicates(), 
#to specify we only want to consider the IMDb ID, use subset, and set inplace equal to True so that the operation
#is performed on the selected dataframe

wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

7076
7033


,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Literally,Cantonese,Chinese,Yiddish,Arabic,Romanized,Russian,Hebrew,Polish,imdb_id
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018


In [187]:
#Let's make a list of columns that have less than 90% null values and use those to trim down our dataset.
[column for column in wiki_movies_df.columns 
 if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
#That will give us the columns that we want to keep
wiki_columns_to_keep=[column for column in wiki_movies_df.columns 
 if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

In [188]:
len(wiki_columns_to_keep)

24

In [189]:
#######################################################################################################################

In [190]:
#8.3.8 Make a Plan to Convert and Parse the Data
#The Wikipedia data is now structured in tabular form, but Britta needs it to have the right data types 
#once it's in the SQL table. For example, we can't do analysis on numeric data if it's stored as a string 
#in the SQL table—it needs to be stored in numeric format. Some of the data also has numeric information 
#written out (like the word "million"). To convert those columns to numbers, the data needs to be parsed.

In [191]:
#First we need to identify which columns need to be converted using .dtypes
wiki_movies_df.dtypes
#Box office should be numeric.
#Budget should be numeric.
#Release date should be a date object.
#Running time should be numeric.

#boxoffice:
box_office=wiki_movies_df['Box office'].dropna()
#As a habit, always check the number of data points that exist after you drop any data. 
#Here we have 5,485 movies with box office data. Does that seem like a reasonable amount? 

In [192]:
#Regular expressions only work on strings, so we'll need to make sure all of the box office data is entered as string
#by using map() method, we can see which values are not strings. first let'a make a is_not_a_string() function:
def is_not_a_string(x):
    return type(x)!=str
box_office[box_office.map(is_not_a_string)]

34                           [US$, 4,212,828]
54      [$6,698,361 (, United States, ), [2]]
74                    [$6,488,144, (US), [1]]
126                [US$1,531,489, (domestic)]
130                          [US$, 4,803,039]
                        ...                  
6980               [$99.6, million, [4], [5]]
6994                   [$365.6, million, [1]]
6995                         [$53.8, million]
7015                     [$435, million, [7]]
7048                   [$529.3, million, [4]]
Name: Box office, Length: 135, dtype: object

In [193]:
#From the output, we can see that there are quite a few data points that are stored as lists. 
#There is a join() string method that concatenates list items into one string; however, we can't just type 
#join(some_list) because the join() method belongs to string objects. We need to make a separator string and 
#then call the join() method on it. For example, the code would be:

In [194]:
#ex:
some_list=['One','Two','Three']
'Mississipi'.join(some_list)

'OneMississipiTwoMississipiThree'

In [195]:
#We'll use a simple space as our joining character and apply the join() function only when our data points are lists. 
#The code looks like the following:
box_office=box_office.apply(lambda x:' '.join(x) if type(x)==list else x)

In [196]:
#Looking through the data, many of the box office numbers are written either like 
#"\$123.4 million" (or billion) or "\$123,456,789." We'll use regular expressions to find out just 
#how many of each style are in our data.
#There is a built-in Python module for regular expressions: re. 
#We'll need to import that library, so add the line below to the first cell, with the other import statements, 
#and rerun the cell.
#import re
#let's import re at the top of our code

In [ ]:
#######################################################################################################################

In [ ]:
#8.3.9 Write Regular Expressions

In [197]:
#Regular expressions only work on strings
#They are used to test if strings are in a specific format or contain a substring in a specific format, 
#to extract pertinent information from strings while discarding unnecessary information, 
#and to perform complicated replacements of substrings

#Each character in a regular expression serves a purpose, based on what kind of character it is. 
#We'll break down the different character types and the purposes they serve:
    ##Literal Characters:
    
#8.3.9 & 8.3.10 SHOULD BE REREAD


In [198]:
#######################################################################################################################

In [199]:
#8.3.10 Parse the box office data

In [200]:
#step1: What is the regular expression that matches a dollar sign?: \$

In [201]:
#step2:Add an arbitrary (but non-zero) number of digits:
#\d:specify digits only
#+: capture one or more digits
#Our RegEx: '\$\d+'

In [202]:
#step3:Add an optimal decimal point
#'\$\d+\n.?'

In [203]:
#step4:Add an arbitrary (but possibly zero) number of more digits
#once again we'll use \d character to specift digits only, but now with the * modifier because there may be no more
#difgirs after the decimal point
#"\$\d+\.?\d*"

In [204]:
#step5:Add a space (maybe more than one):
#\s character is used to match whitespaces characters. and we'll use * modifier to be safe because there may be more.
#"\$\d+\.?\d*\s*" 

In [205]:
#step6:add the word million or billion
#"\$\d+\.?\d*\s*[mb]illion"

In [206]:
#Create a variable form_one and set it equal to the finished regular expression string. 
#Because we need the escape characters to remain, we need to preface the string with an r
form_one = r'\$\d+\.?\d*\s*[mb]illion'

In [207]:
#let's count up how many box office values match our first form
#use str.contains()
#to ignore upper/lowercase, add flags=re.IGNORECASE, in case data is not a string: na=False
#Finally we'll call the sum
box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()
#There are 3,896 box office values that match the form "$123.4 million/billion."

3896

In [208]:
#Let's create the Second Form Now:
form_two = r'\$\d{1,3}(?:,\d{3})+'
box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()
#There are 1,544 box office values that match the form "$123,456,789."

1544

In [209]:
#To make our code easier to understand, we'll create two Boolean Series called matches_form_one 
#and matches_form_two, and then select the box office values that don't match either
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE, na=False)

In [210]:
# this will throw an error!
#box_office[(not matches_form_one) and (not matches_form_two)]
#instead:
box_office[~matches_form_one & ~matches_form_two] #` instead of not, & instead of and, | instead of or

34                         US$ 4,212,828
79                              $335.000
110                   $4.35-4.37 million
130                        US$ 4,803,039
600                           $5000 (US)
731                         $ 11,146,270
957                             $ 50,004
1070                          35,254,617
1147    $ 407,618 (U.S.) (sub-total) [1]
1446                        $ 11,829,959
1480                          £3 million
1611                            $520.000
1865                        ¥1.1 billion
2032                                 N/A
2091                                $309
2130               US$ 171.8 million [9]
2257                   US$ 3,395,581 [1]
2263            $ 1,223,034 ( domestic )
2347                            $282.175
2638            $ 104,883 (US sub-total)
2665         926,423 admissions (France)
2697      $ 1.7 million (US) (sub-total)
2823                            $414.000
2924                            $621.000
3088           $

In [211]:
# Some values have spaces in between the dollar sign and the number
#This is easy to fix. Just add \s* after the dollar signs. The new forms should look like the following:
form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
form_two = r'\$\s*\d{1,3}(?:,\d{3})+'

In [212]:
#Some values use a period as a thousands separator, not a comma
#Simply change form_two to allow for either a comma or period as a thousands separator. 
#We’d ordinarily do that by putting the comma and period inside straight brackets [,.], 
#but the period needs to be escaped with a slash [,\.]. The code should match the following
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+'

In [213]:
#The results will also match values like 1.234 billion, but we're trying to change raw numbers like $123.456.789. 
#We don't want to capture any values like 1.234 billion, so we need to add a negative lookahead group that looks 
#ahead for "million" or "billion" after the number and rejects the match if it finds those strings. 
#Don't forget the space! The new form should look like this:
form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

In [214]:
#Some values are given as a range. 
#To solve this problem, we'll search for any string that starts with a dollar sign and ends with a hyphen, 
#and then replace it with just a dollar sign using the replace() method
#We can use regular expressions in the first argument by sending the parameter regex=True, as shown below.
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [215]:
#"Million" is sometimes misspelled as "millon."
#This is easy enough to fix; we can just make the second "i" optional in our match string with a 
#question mark as follows
form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'